In [ ]:
'''
Dataloader
'''
# !wget https://zenodo.org/record/1489920/files/articles-training-bypublisher-20181122.zip?download=1
# !unzip bypublisher.zip
# !wget https://zenodo.org/record/1489920/files/ground-truth-training-bypublisher-20181122.zip?download=1
# !unzip gd.zip

# import sys
# import xml.sax
# import xml.parsers.expat
# import time 
# import pickle
# import os
# from sklearn.svm import LinearSVC
# from sklearn.feature_extraction.text import TfidfVectorizer 
# from collections import OrderedDict
# class ParseHandler(xml.sax.handler.ContentHandler):
#     def __init__(self):
#         self.text=""
#     def startElement(self, name, attrs):
#         self.CurrentData = name
#         if(self.CurrentData=="article"):
#             labels.append(attrs['hyperpartisan'])


# labels = []
# parser = xml.sax.make_parser()
# parser.setFeature(xml.sax.handler.feature_namespaces, 0) 
# Handler = ParseHandler()
# parser.setContentHandler(Handler)
# parser.parse("gd.xml")

# text=[]
# import re
# class ParseHandler1(xml.sax.handler.ContentHandler):
#     def __init__(self):
#         self.text=""
#         self.r1=0
#     def startElement(self, name, attrs):
#         self.CurrentData = name
#     def endElement(self, name):
#         if(name=="article"):
#             self.text=clean_text(self.text)
#             self.text=re.sub(r'([^\s\w]|_)+', '', self.text)
#             self.text=self.text.strip()
#             text.append(self.text)
#             self.text=""    
#     def characters(self, data):
#         self.text += data

# def clean_text(text1):
#     text1 = text1.replace(".", ". ")
#     text1 = text1.replace(" _", " ")
#     text1 = text1.replace("  ", " ")
#     text1 = text1.replace(". . . .", "...")
#     text1 = text1.replace(". . .", "...")
#     return text1

# parser = xml.sax.make_parser()
# parser.setFeature(xml.sax.handler.feature_namespaces, 0) 
# Handler = ParseHandler1()
# parser.setContentHandler(Handler)
# parser.parse("bypublisher.xml")

# import pandas as pd 
# df1 = pd.DataFrame(text)
# df2=pd.DataFrame(labels)
# # print(df1)
# result = pd.concat([df1, df2], axis=1, sort=False) 
# result.columns = ['texts','labels']
# result.to_csv('publisher.csv')
# cp 'drive/My Drive/ANLP_proj/publisher.csv' .

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
max_articles = 50000
durl = 'drive/My Drive/ANLP_proj/'
articles = []
labels = []
csv_File = durl + 'publisher.csv'
df_itr = pd.read_csv(csv_File,chunksize=100)
cur = 0
for it in df_itr:
  if cur >= max_articles//100:
    break
  articles.extend(it['texts'])
  labels.extend(it['labels'])
  cur+=1
  # for row in it:
    # print(row)
    # articles.extend(row.iloc[cur]['texts'])
    # labels.extend(row.iloc[cur]['labels'])
labels = np.array(labels,dtype = np.uint8)
print('number of samples = ',len(labels), labels[1])

number of samples =  50000 1


In [ ]:
# df = pd.read_csv('article.csv')
# articles = df['texts'].to_numpy()
# labels = df['labels'].to_numpy()
# print(articles.shape)


In [ ]:
!pip install transformers
import tensorflow as tf
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
max_length_test = 50
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(posts,labels):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []
  for post, label in zip(posts,labels):
    if not isinstance(post, str):
      continue
    bert_input = tokenizer.encode_plus(post,
                                       truncation=True,           
                                      add_special_tokens = True, # add [CLS], [SEP]
                                      max_length = max_length_test, # max length of the text that can go to BERT
                                      padding='max_length', # add [PAD] tokens
                                      return_attention_mask = True, # add attention mask to not focus on pad token

                                      )
  
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

def convert_to_tfdataset(articles,labels,batch_size = 30):
  dataset = encode_examples(articles,labels)
  dataset = dataset.shuffle(dataset.__len__().numpy()).batch(batch_size)
  return dataset


In [ ]:
# split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( articles, labels, test_size=0.3, random_state=21)
train_dataset = convert_to_tfdataset(X_train,y_train,batch_size = 30)
test_dataset = convert_to_tfdataset(X_test,y_test,batch_size = 30)

In [ ]:
from transformers import BertConfig
model_name = 'bert-base-uncased'
config1 = BertConfig.from_pretrained(model_name)
config1.num_labels = 2
from transformers import TFBertForSequenceClassification

In [ ]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained(model_name,config = config1)
learning_rate = 2e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_history = model.fit(train_dataset,epochs=5,verbose=1,validation_data=test_dataset)

Epoch 1/5
1167/1167 [==============================] - 440s 377ms/step - loss: 0.3859 - accuracy: 0.8176 - val_loss: 0.3435 - val_accuracy: 0.8449
Epoch 2/5
1167/1167 [==============================] - 454s 389ms/step - loss: 0.2544 - accuracy: 0.8892 - val_loss: 0.3112 - val_accuracy: 0.8630
Epoch 3/5
1167/1167 [==============================] - 455s 390ms/step - loss: 0.1434 - accuracy: 0.9440 - val_loss: 0.3879 - val_accuracy: 0.8628
Epoch 4/5
1167/1167 [==============================] - 454s 389ms/step - loss: 0.0718 - accuracy: 0.9749 - val_loss: 0.4562 - val_accuracy: 0.8635
Epoch 5/5
1167/1167 [==============================] - 454s 389ms/step - loss: 0.0420 - accuracy: 0.9853 - val_loss: 0.5676 - val_accuracy: 0.8605
